In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches

# Đặc trưng 
feature_names = ['mean_x', 'mean_y', 'mean_z',
                 'mean_x2', 'mean_y2', 'mean_z2',
                 'rms_x', 'rms_y', 'rms_z',
                 'rms_x2', 'rms_y2', 'rms_z2',
                 'std_x', 'std_y', 'std_z',
                 'std_x2', 'std_y2', 'std_z2',
                'med_x', 'med_y', 'med_z',
                'med_x2', 'med_y2', 'med_z2']

# Hành vi
target_names = ['Feeding', 'Lying', 'Standing', 'Normal Walking']

def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    Trả về 1 ma trận xoay theo chiều ngược kim đồng hồ quanh trục được xác định với vecto axis
    và 1 góc quay theta radian.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])
    
    
def featuresFromBuffer(at):
    feat = np.zeros(24)
    
    # Gia tốc ở chân
    x = np.array(at.iloc[:,0], dtype=np.float64)
    y = np.array(at.iloc[:,1], dtype=np.float64)
    z = np.array(at.iloc[:,2], dtype=np.float64)
    
    # Gia tốc ở cổ
    x2 = np.array(at.iloc[:,3], dtype=np.float64)
    y2 = np.array(at.iloc[:,4], dtype=np.float64)
    z2 = np.array(at.iloc[:,5], dtype=np.float64)
      
    # Giá trị trung bình 
    means = [np.mean(i) for i in [x, y, z]]
    feat[0:3] = means
    means2 = [np.mean(i) for i in [x2, y2, z2]]
    feat[3:6] = means2
    
    # Giá trị hiệu dụng (Root mean square)
    rms = [np.sqrt(np.mean(i**2)) for i in [x, y, z]]
    feat[6:9] = rms
    rms2 = [np.sqrt(np.mean(i**2)) for i in [x2, y2, z2]]
    feat[9:12] = rms2
    
    # Standard deviation
    # Độ lệch chuẩn 
    std = [np.std(i) for i in [x, y, z]]
    feat[12:15] = std
    std2 = [np.std(i) for i in [x2, y2, z2]]
    feat[15:18] = std2 
    
    # Median
    # Giá trị trung vị
    med = [np.median(i) for i in [x, y, z]]
    feat[18:21] = med
    med2 = [np.median(i) for i in [x2, y2, z2]]
    feat[21:24] = med2
        
    return feat
    

In [2]:
# Import data

df = pd.read_excel('3CowData_2sensor.xlsx')


# Mục mới

In [3]:
# Prepare data
# Load dữ liệu từ file excel lên

Feeding = df[['Feeding', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']]
Feeding = Feeding.dropna()
Feeding.index = pd.RangeIndex(len(Feeding.index))
Feeding = Feeding.drop(0)

axis = [1, 0, 0]
#lech 10 do
#theta = 0.174 

#lech 30 do
theta = 0.523

#lech 45 do
#theta = 0.785

for num in range(1, len(Feeding['Unnamed: 3'])%1):
  fed = np.dot(rotation_matrix(axis, theta), [Feeding['Unnamed: 3'][num], Feeding['Unnamed: 4'][num], Feeding['Unnamed: 5'][num]])
  Feeding['Unnamed: 3'][num] = fed[0]
  Feeding['Unnamed: 4'][num] = fed[1]
  Feeding['Unnamed: 5'][num] = fed[2]


Lying = df[['Lying', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11']]
Lying = Lying.dropna()
Lying.index = pd.RangeIndex(len(Lying.index))
Lying = Lying.drop(0)
for num in range(1, len(Lying['Unnamed: 9'])%1):
  fed = np.dot(rotation_matrix(axis, theta), [Lying['Unnamed: 9'][num], Lying['Unnamed: 10'][num], Lying['Unnamed: 11'][num]])
  Lying['Unnamed: 9'][num] = fed[0]
  Lying['Unnamed: 10'][num] = fed[1]
  Lying['Unnamed: 11'][num] = fed[2]

Standing = df[['Standing', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17']]
Standing = Standing.dropna()
Standing.index = pd.RangeIndex(len(Standing.index))
Standing = Standing.drop(0)
for num in range(1, len(Standing['Unnamed: 15'])%1):
  fed = np.dot(rotation_matrix(axis, theta), [Standing['Unnamed: 15'][num], Standing['Unnamed: 16'][num], Standing['Unnamed: 17'][num]])
  Standing['Unnamed: 15'][num] = fed[0]
  Standing['Unnamed: 16'][num] = fed[1]
  Standing['Unnamed: 17'][num] = fed[2]

Normal_walking = df[['Normal walking', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23']]
Normal_walking = Normal_walking.dropna()
Normal_walking.index = pd.RangeIndex(len(Normal_walking.index))
Normal_walking = Normal_walking.drop(0)
for num in range(1, len(Normal_walking['Unnamed: 21'])%1):
  fed = np.dot(rotation_matrix(axis, theta), [Normal_walking['Unnamed: 21'][num], Normal_walking['Unnamed: 22'][num], Normal_walking['Unnamed: 23'][num]])
  Normal_walking['Unnamed: 21'][num] = fed[0]
  Normal_walking['Unnamed: 22'][num] = fed[1]
  Normal_walking['Unnamed: 23'][num] = fed[2]

In [4]:
def prepare_data(file_name):
    
    # Import data
    df = pd.read_excel(file_name)
    
    # Vecto, góc quay
    axis = [1, 0, 0]
    theta = 0.523     #lech 30 do

    # Xử lý dữ liệu
    Feeding = df[['Feeding', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']]
    Feeding = Feeding.dropna()
    Feeding.index = pd.RangeIndex(len(Feeding.index))
    Feeding = Feeding.drop(0)
    for num in range(1, len(Feeding['Unnamed: 3'])%1):
        fed = np.dot(rotation_matrix(axis, theta), [Feeding['Unnamed: 3'][num], Feeding['Unnamed: 4'][num], Feeding['Unnamed: 5'][num]])
        Feeding['Unnamed: 3'][num] = fed[0]
        Feeding['Unnamed: 4'][num] = fed[1]
        Feeding['Unnamed: 5'][num] = fed[2]

    Lying = df[['Lying', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11']]
    Lying = Lying.dropna()
    Lying.index = pd.RangeIndex(len(Lying.index))
    Lying = Lying.drop(0)
    for num in range(1, len(Lying['Unnamed: 9'])%1):
        fed = np.dot(rotation_matrix(axis, theta), [Lying['Unnamed: 9'][num], Lying['Unnamed: 10'][num], Lying['Unnamed: 11'][num]])
        Lying['Unnamed: 9'][num] = fed[0]
        Lying['Unnamed: 10'][num] = fed[1]
        Lying['Unnamed: 11'][num] = fed[2]

    Standing = df[['Standing', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17']]
    Standing = Standing.dropna()
    Standing.index = pd.RangeIndex(len(Standing.index))
    Standing = Standing.drop(0)
    for num in range(1, len(Standing['Unnamed: 15'])%1):
        fed = np.dot(rotation_matrix(axis, theta), [Standing['Unnamed: 15'][num], Standing['Unnamed: 16'][num], Standing['Unnamed: 17'][num]])
        Standing['Unnamed: 15'][num] = fed[0]
        Standing['Unnamed: 16'][num] = fed[1]
        Standing['Unnamed: 17'][num] = fed[2]

    Normal_walking = df[['Normal walking', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23']]
    Normal_walking = Normal_walking.dropna()
    Normal_walking.index = pd.RangeIndex(len(Normal_walking.index))
    Normal_walking = Normal_walking.drop(0)
    for num in range(1, len(Normal_walking['Unnamed: 21'])%1):
        fed = np.dot(rotation_matrix(axis, theta), [Normal_walking['Unnamed: 21'][num], Normal_walking['Unnamed: 22'][num], Normal_walking['Unnamed: 23'][num]])
        Normal_walking['Unnamed: 21'][num] = fed[0]
        Normal_walking['Unnamed: 22'][num] = fed[1]
        Normal_walking['Unnamed: 23'][num] = fed[2]
        
    
    
    
    # thực hiện windowing dữ liệu
    # chia mỗi khối dữ liệu 16s
    window_size = 16
    stride = 6

    train_ratio = 0.6

    X_feeding_train = [Feeding[i:i+window_size] for i in range(0, int(len(Feeding)*train_ratio), stride) if i+window_size<=int(len(Feeding))]
    X_feeding_test = [Feeding[i:i+window_size] for i in range(int(len(Feeding)*train_ratio), len(Feeding), stride) if i+window_size<=len(Feeding)]

    X_lying_train = [Lying[i:i+window_size] for i in range(0, int(len(Lying)*train_ratio), stride) if i+window_size<=int(len(Lying))]
    X_lying_test = [Lying[i:i+window_size] for i in range(int(len(Lying)*train_ratio), len(Lying), stride) if i+window_size<=len(Lying)]

    X_standing_train = [Standing[i:i+window_size] for i in range(0, int(len(Standing)*train_ratio), stride) if i+window_size<=int(len(Standing))]
    X_standing_test = [Standing[i:i+window_size] for i in range(int(len(Standing)*train_ratio), len(Standing), stride) if i+window_size<=len(Standing)]

    X_normalw_train = [Normal_walking[i:i+window_size] for i in range(0, int(len(Normal_walking)*train_ratio), stride) if i+window_size<=int(len(Normal_walking))]
    X_normalw_test = [Normal_walking[i:i+window_size] for i in range(int(len(Normal_walking)*train_ratio), len(Normal_walking), stride) if i+window_size<=len(Normal_walking)]

    
    
    
    # Catch 'em all
    my_train_data = []
    my_train_label = []

    my_test_data = []
    my_test_label = []

    for acts in X_feeding_train:
        my_train_data.append(acts)
        my_train_label.append(0)

    for acts in X_lying_train:
        my_train_data.append(acts)
        my_train_label.append(1)
        
    for acts in X_standing_train:
        my_train_data.append(acts)
        my_train_label.append(2)  

    for acts in X_normalw_train:
        my_train_data.append(acts)
        my_train_label.append(3)
    
    #Catch 'em all
    for acts in X_feeding_test:
        my_test_data.append(acts)
        my_test_label.append(0)

    for acts in X_lying_test:
        my_test_data.append(acts)
        my_test_label.append(1)
        
    for acts in X_standing_test:
        my_test_data.append(acts)
        my_test_label.append(2)

    for acts in X_normalw_test:
        my_test_data.append(acts)
        my_test_label.append(3)
    
    
    
    
    # Trích xuất đặc trưng
    train_features = []
    test_features = []
    
    for action in my_train_data:
        feat = featuresFromBuffer(action)
        train_features.append(feat)
    for action in my_test_data:
        feat = featuresFromBuffer(action)
        test_features.append(feat)
    
    return train_features, test_features, my_train_label, my_test_label
    

def model(train_features, test_features, my_train_label, my_test_label):
    # Train model
    rf = RandomForestClassifier(n_estimators=25, random_state=0).fit(train_features, my_train_label)

    # Ma trận nhầm lẫn
    predicted = rf.predict(test_features)

    confusion_test = confusion_matrix(my_test_label, predicted)

    class_labels = ['Feeding', 'Lying', 'Standing', 'Walking']
    c_matrix = pd.DataFrame(confusion_test, index=class_labels, columns=class_labels)

    # Thêm cột total
    c_matrix['Total'] = c_matrix.sum(axis=1)

    # Thêm hàng total
    total_row = c_matrix.sum(axis=0).to_frame().T
    total_row.index = ['Total']
    c_matrix = pd.concat([c_matrix, total_row])

    # Hành vi được phân loại
    classified_behaviors = c_matrix.loc[class_labels, 'Total'].values

    # Hành vi thực tế
    actual_behaviors = c_matrix.loc['Total', class_labels].values

    # So sánh kết quả
    behavior_df = pd.DataFrame([classified_behaviors, actual_behaviors], 
                               index=['Classified', 'Actual'], 
                               columns=class_labels)
    
    return c_matrix, behavior_df

def plot_labelled_scatter(X, y, class_labels):

    # Visualisation: t-sne
    X_min = np.array(X[:, 0])
    X_max = np.array(X[:, 1])
    
    num_labels = len(class_labels)

    color_array = ['#000000', '#00AAFF', '#FF00AA', '#00FF4C']
    cmap_bold = ListedColormap(color_array)
    bnorm = BoundaryNorm(np.arange(0, num_labels+1, 1), ncolors=num_labels)
    plt.figure(figsize=(12, 12))

    plt.scatter(X_min, X_max, s=65, c=y, cmap=cmap_bold, norm = bnorm, alpha = 0.40, lw=1)

    h = []
    for c in range(0, num_labels):
        h.append(mpatches.Patch(color=color_array[c], label=class_labels[c]))
    plt.legend(handles=h)
    
    plt.xlabel('First t-SNE feature')
    plt.ylabel('Second t-SNE feature')
    plt.title('Cow ID t-SNE, DATASET')

    plt.show()
    
    

In [5]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QPushButton, QLineEdit,QFileDialog,
                              QDateEdit, QDialog, QVBoxLayout, QTextBrowser, QGridLayout, QMessageBox) 
from PyQt5.QtCore import QFileInfo, QDate
import numpy as np
from sklearn.manifold import TSNE
import matplotlib
matplotlib.use('Qt5Agg')

class UploadDataApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('Classify the behavior of the cow')
        self.setGeometry(200, 200, 430, 200)

        layout = QVBoxLayout()

        grid_layout = QGridLayout()

        # Label và ô nhập ngày
        self.date_label = QLabel('Date:')
        self.date_edit = QDateEdit()
        self.date_edit.setCalendarPopup(True)
        self.date_edit.setDate(QDate.currentDate())

        grid_layout.addWidget(self.date_label, 0, 0)
        grid_layout.addWidget(self.date_edit, 0, 1, 1, 2)

        # Label và ô nhập đường dẫn file Excel
        self.file_label = QLabel('Excel File:')
        self.file_edit = QLineEdit()
        self.file_button = QPushButton('Browse')
        self.file_button.clicked.connect(self.browse_file)

        grid_layout.addWidget(self.file_label, 1, 0)
        grid_layout.addWidget(self.file_edit, 1, 1)
        grid_layout.addWidget(self.file_button, 1, 2)

        # Nút Upload Data
        self.upload_button = QPushButton('Upload Data')
        self.upload_button.clicked.connect(self.show_data_window)

        layout.addLayout(grid_layout)
        layout.addWidget(self.upload_button)

        self.setLayout(layout)

    def browse_file(self):
        file_path, _ = QFileDialog.getOpenFileName(self, 'Open Excel File', '', 'Excel Files (*.xlsx *.xls)')
        if file_path:
            self.file_edit.setText(file_path)

    def show_data_window(self):
        # Lấy dữ liệu từ các trường nhập
        selected_date = self.date_edit.date().toString('dd-MM-yyyy')
        file_path = self.file_edit.text()

        if not file_path:
            # Hiển thị cảnh báo nếu chưa có file được chọn
            QMessageBox.warning(self, "Warning", "Please select an Excel file.")
            return
        
        # Tách tên tệp Excel từ đường dẫn tệp
        file_info = QFileInfo(file_path)
        file_name = file_info.fileName()

        # Tạo cửa sổ mới để hiển thị dữ liệu
        data_window = QDialog(self)
        data_window.setWindowTitle('Data Display')
        data_window.setGeometry(400, 400, 800, 600) 

        # Hiển thị dữ liệu trong cửa sổ mới
        layout = QVBoxLayout(data_window)
        
        train_features, test_features, my_train_label, my_test_label = prepare_data(file_name)
        c_matrix, behavior_df  = model(train_features, test_features, my_train_label, my_test_label)    
        
        # Tạo nội dung cho QTextBrowser
        content = f"<h3>Uploaded Date: {selected_date}</h3> \n"
        content += f"<h3>File Name: {file_name}</h3>"
        content += f"<h3>Confusion Matrix:</h3><pre>{str(c_matrix)}</pre> \n"
        content += f"<h3>Number of Behaviors:</h3><pre>{str(behavior_df)}</pre> \n"
        
        # Hiển thị nội dung trong QTextBrowser
        text_browser = QTextBrowser(data_window)
        text_browser.insertHtml(content)
        layout.addWidget(text_browser)
        
        # Hiển thị biểu đồ
        X_visual = np.array(train_features)
        y_visual = np.array(my_train_label, dtype=np.uint8)
        X_tsne = TSNE(random_state=1000).fit_transform(X_visual)
        class_labels = ['Feeding', 'Lying', 'Standing', 'Walking']
        plot_labelled_scatter(X_tsne, y_visual, class_labels)
        
        data_window.exec_()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = UploadDataApp()
    window.show()
    sys.exit(app.exec_())


SystemExit: 0

c:\Users\Tran Giang Lam\Documents\code_cow\code_cow\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
